<a href="https://colab.research.google.com/github/thoongee/TextAnalysis/blob/main/coupang_crawling_0428.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openpyxl
!pip install requests
!pip install bs4
!pip install lxml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=cc219bfd5123c15f932995e8de8586eda0d2dfe73fba089477a370c45cdcc2a0
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from bs4 import BeautifulSoup as bs
from pathlib import Path
from typing import Optional,Union,Dict,List
from openpyxl import Workbook
import time
import os
import re
import requests as rq
import json
import csv
import numpy as np

import requests, openpyxl

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def get_headers(
    key: str,
    default_value: Optional[str] = None
    )-> Dict[str,Dict[str,str]]:
    """ Get Headers """
    JSON_FILE : str = '/content/drive/MyDrive/텍스트분석2023/headers.json'

    with open(JSON_FILE,'r',encoding='UTF-8') as file:
        headers : Dict[str,Dict[str,str]] = json.loads(file.read())

    try :
        return headers[key]
    except:
        if default_value:
            return default_value
        raise EnvironmentError(f'Set the {key}')

class Coupang:
    @staticmethod
    def get_product_code(url: str)-> str:
        """ 입력받은 URL 주소의 PRODUCT CODE 추출하는 메소드 """
        prod_code : str = url.split('products/')[-1].split('?')[0]
        return prod_code

    def __init__(self)-> None:
        self.__headers : Dict[str,str] = get_headers(key='headers')
        self.page = 1
        self.star5 = 0
        self.star3 = 0
        self.star1 = 0

    ## review_url 파라미터 추가
    def main(self,review_url)-> List[List[Dict[str,Union[str,int]]]]:
        # URL 주소
        URL : str = self.input_review_url(review_url)

        # URL의 Product Code 추출
        prod_code : str = self.get_product_code(url=URL)

        url = [f'https://www.coupang.com/vp/product/reviews?productId={prod_code}&page=1&size=5&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=3&ratingSummary=true']
        self.__headers['referer'] = URL
        with rq.Session() as session:
            with session.get(url=url[0],headers=self.__headers) as response :
                html = response.text
                soup = bs(html,'html.parser')
                test = soup.select("div.js_reviewArticleHiddenValue")
                total_review = 0
                for i in range(len(test)):
                    total_review += int(test[i].attrs['data-count'])
                total_page = int(np.ceil(total_review/5))
                print("(이야야야야ㅏㅇ) ", total_review, int(np.ceil(total_review/5)))

        # URL 주소 재가공
        URLS : List[str] = [f'https://www.coupang.com/vp/product/reviews?productId={prod_code}&page={page}&size=5&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=3&ratingSummary=true' for page in range(1,self.input_page_count() + 1)]

        # __headers에 referer 키 추가
        self.__headers['referer'] = URL

        with rq.Session() as session:
            return [self.fetch(url=url,session=session) for url in URLS]

    def fetch(self,url:str,session)-> List[Dict[str,Union[str,int]]]:
        
        save_data : List[Dict[str,Union[str,int]]] = list()

        with session.get(url=url,headers=self.__headers) as response :
            html = response.text
            soup = bs(html,'html.parser')

            # Article Boxes
            article_lenth = len(soup.select('article.sdp-review__article__list'))

            for idx in range(article_lenth):
                dict_data : Dict[str,Union[str,int]] = dict()
                articles = soup.select('article.sdp-review__article__list')

                review_content = articles[idx].select_one('div.sdp-review__article__list__review > div')
                if review_content == None:
                    # review_content = '등록된 리뷰내용이 없습니다'
                    continue
                else:
                    review_content = re.sub('[\n\t]','',review_content.text.strip())

                rating = articles[idx].select_one('div.sdp-review__article__list__info__product-info__star-orange')
                # print("(3) ", rating)
                if int(rating.attrs['data-rating']) == 1:
                    if self.star1 >= 50:
                        continue
                    elif self.star1 < 50:
                        rating, user_name, prod_name, headline, help_cnt = self.sub(articles, idx, 1)
                        self.star1 += 1
                elif int(rating.attrs['data-rating']) == 3:
                    if self.star3 >= 50:
                        continue
                    elif self.star3 < 50:
                        rating, user_name, prod_name, headline, help_cnt = self.sub(articles, idx, 3)
                        self.star3 += 1
                elif int(rating.attrs['data-rating']) == 5:
                    if self.star5 >= 50:
                        continue
                    elif self.star5 < 50:
                        rating, user_name, prod_name, headline, help_cnt = self.sub(articles, idx, 5)
                        self.star5 += 1
                else:
                    continue
               

                # 구매자 이름
                user_name = articles[idx].select_one('span.sdp-review__article__list__info__user__name')
                if user_name == None or user_name.text == '':
                    user_name = '-'
                else:
                    user_name = user_name.text.strip()

                # 평점
                rating = articles[idx].select_one('div.sdp-review__article__list__info__product-info__star-orange')
                if rating == None:
                    rating = 0
                else :
                    rating = int(rating.attrs['data-rating'])

                # 구매자 상품명
                prod_name = articles[idx].select_one('div.sdp-review__article__list__info__product-info__name')
                if prod_name == None or prod_name.text == '':
                    prod_name = '-'
                else:
                    prod_name = prod_name.text.strip()

                # 헤드라인(타이틀)
                headline = articles[idx].select_one('div.sdp-review__article__list__headline')
                if headline == None or headline.text == '':
                    headline = '등록된 헤드라인이 없습니다'
                else:
                    headline = headline.text.strip()

                # 리뷰 내용
                review_content = articles[idx].select_one('div.sdp-review__article__list__review > div')
                if review_content == None :
                    review_content = '등록된 리뷰내용이 없습니다'
                else:
                    review_content = re.sub('[\n\t]','',review_content.text.strip())

                # 맛 만족도
                answer = articles[idx].select_one('span.sdp-review__article__list__survey__row__answer')
                if answer == None or answer.text == '':
                    answer = '맛 평가 없음'
                else:
                    answer = answer.text.strip()

                dict_data['prod_name'] = prod_name
                dict_data['user_name'] = user_name
                dict_data['rating'] = rating
                dict_data['headline'] = headline
                dict_data['review_content'] = review_content
                dict_data['answer'] = answer

                save_data.append(dict_data)

                print(dict_data , '\n')

            time.sleep(1)

            return save_data
    def sub(self, articles, idx, rat):
        # print()
        # if star_count >= 50:
        #     return "x", "x", "x", "x", "x"
        # else:
        rating = rat

        # 구매자 이름
        user_name = articles[idx].select_one('span.sdp-review__article__list__info__user__name')
        if user_name == None or user_name.text == '':
            user_name = '-'
        else:
            user_name = user_name.text.strip()

        # 평점
        # rating = articles[idx].select_one('div.sdp-review__article__list__info__product-info__star-orange')
        # if rating == None:
        #     rating = 0
        # else :
        #     rating = int(rating.attrs['data-rating'])

        # 구매자 상품명
        prod_name = articles[idx].select_one('div.sdp-review__article__list__info__product-info__name')
        if prod_name == None or prod_name.text == '':
            prod_name = '-'
        else:
            prod_name = prod_name.text.strip()

        # 헤드라인(타이틀)
        headline = articles[idx].select_one('div.sdp-review__article__list__headline')
        if headline == None or headline.text == '':
            headline = '등록된 헤드라인이 없습니다'
        else:
            headline = headline.text.strip()

        # 리뷰 내용
        # review_content = articles[idx].select_one('div.sdp-review__article__list__review > div')
        # if review_content == None :
        #     review_content = '등록된 리뷰내용이 없습니다'
        # else:
        #     review_content = re.sub('[\n\t]','',review_content.text.strip())

        # 도움횟수
        help_cnt = articles[idx].select_one('div.sdp-review__article__list__help.js_reviewArticleHelpfulContainer')
        if help_cnt == None:
            help_cnt = 0
        else:
            help_cnt = int(help_cnt.attrs['data-count'])

        return rating, user_name, prod_name, headline, help_cnt

    def input_review_url(self,review_url)-> str:
        while True:
            # Window
            os.system('cls')
            
            # Review URL
#             review_url : str = input('원하시는 상품의 URL 주소를 입력해주세요\n\nEx)\nhttps://www.coupang.com/vp/products/6451503812?itemId=14007944553&vendorItemId=73528488680&sourceType=srp_product_ads&clickEventId=28aaab30-71e3-4f30-9059-07a29eb1b27f&korePlacement=15&koreSubPlacement=6&q=%EB%9E%A9%EB%85%B8%EC%89%AC&itemsCount=36&searchId=af6bda06076947a39f847ed86a718c34&rank=5&isAddedCart=\n\n:')
            if not review_url :
                # Window
                os.system('cls')
                # Mac
                #os.system('clear')
                print('URL 주소가 입력되지 않았습니다')
                continue
            return review_url

    def input_page_count(self)-> int:
        # Window
        os.system('cls')
        # Mac
        #os.system('clear')
        while True:
            page_count : str = input('페이지 수를 입력하세요\n\n:')
            if not page_count:
                print('페이지 수가 입력되지 않았습니다\n')
                continue

            return int(page_count)

class OpenPyXL:
    @staticmethod
    def save_file(review_url)-> None:
        # 크롤링 결과
        results : List[List[Dict[str,Union[str,int]]]] = Coupang().main(review_url)

        wb = Workbook()
        ws = wb.active
        ws.append(['상품명','구매자 이름','구매자 평점','리뷰 제목','리뷰 내용','도움횟수'])

        row = 2

        for x in results:
            for result in x :
                ws[f'A{row}'] = result['prod_name']
                ws[f'B{row}'] = result['user_name']
                ws[f'C{row}'] = result['rating']
                ws[f'D{row}'] = result['headline']
                ws[f'E{row}'] = result['review_content']
                ws[f'F{row}'] = result['answer']

                row += 1
        ## 저장경로는 수정할 것
        savePath : str = os.path.abspath('C:\\Users\\cjina\\OneDrive - 서울과학기술대학교\\대학원\\수업\\텍스트분석\\팀플\\data')
        fileName : str = results[0][0]['prod_name'] + '.xlsx'

        if not os.path.exists(savePath):
            os.mkdir(savePath)

        wb.save(os.path.join(savePath,fileName))
        wb.close()
        print(f'파일 저장완료!\n\n{os.path.join(savePath,fileName)}')





In [11]:
def Evinue():
  #url = page url
    headers = {
        'Referer': 'https://www.google.com/',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'ko,th;q=0.9,en-US;q=0.8,en;q=0.7,ru;q=0.6,ko-KR;q=0.5,la;q=0.4',
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36"
    }
    link_list = []
    page_num = 1
    while(True):
      ## 쿠팡 스킨케어 링크
      url = f'https://www.coupang.com/np/categories/176530?listSize=60&brand=&offerCondition=&filterType=top_brand&isPriceRange=false&minPrice=&maxPrice=&page={page_num}&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=saleCountDesc&filter=&component=176430&rating=0'
      page_num += 1
      res = requests.get(url, headers = headers, verify=False)
      res.raise_for_status()
      soup = bs(res.text, "lxml")

      items = soup.find_all("li", attrs={"class":re.compile("^baby-product")})
      
      for item in items:
          link = item.find("a", attrs={"class":"baby-product-link"})["href"]
          p_link = 'https://www.coupang.com'+link+'&isAddedCart='
          link_list.append(p_link)
          print(len(link_list))
          ## 총 크롤링 할 상품 개수 200개
          if len(link_list) == 10:
            return link_list

def NoEvinue():
    
    headers = {
        'Referer': 'https://www.google.com/',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'ko,th;q=0.9,en-US;q=0.8,en;q=0.7,ru;q=0.6,ko-KR;q=0.5,la;q=0.4',
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36"
    }

    link_list = []
    page_num = 1
    while(True):
      ## 쿠팡 스킨케어 링크
      url= f'https://www.coupang.com/np/categories/176530?listSize=60&brand=&offerCondition=&filterType=top_brand&isPriceRange=false&minPrice=&maxPrice=&page={page_num}&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=saleCountDesc&filter=&component=176430&rating=0'
      page_num += 1
      res = requests.get(url, headers = headers, verify=False)
      res.raise_for_status()
      soup = bs(res.text, "lxml")
      
      items = soup.find_all("li", attrs={"class":re.compile("^baby-product")})
      
      for item in items:
          badge = item.find("img", attrs={"class":"badge-ico"})
          if badge:
              continue
          else:
              link = item.find("a", attrs={"class":"baby-product-link"})["href"]
              p_link = 'https://www.coupang.com'+link+'&isAddedCart='
              link_list.append(p_link)   
          ## 크롤링할 상품 개수 총 200개
          if len(link_list) == 200:
            break

    return link_list

In [ ]:
urls = Evinue()
for url in urls:
    OpenPyXL.save_file(url)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.coupang.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
(이야야야야ㅏㅇ)  19200 3840


In [ ]:
urls = NoEvinue()
for url in urls:
    OpenPyXL.save_file(url)